In [0]:
from pyspark.sql import SQLContext
from pyspark.sql.types import Row, StructField, StructType, StringType, IntegerType
# One way to create a DataFrame is to first define an RDD from a list of Rows 
some_rdd = sc.parallelize([Row(name="John", age=19),
                           Row(name="Smith", age=23),
                           Row(name="Sarah", age=18)])
some_rdd.collect()

Out[25]: [Row(age=19, name='John'),
 Row(age=23, name='Smith'),
 Row(age=18, name='Sarah')]

In [0]:
# The DataFrame is created from the RDD or Rows
# Infer schema from the first row, create a DataFrame and print the schema
some_df = spark.createDataFrame(some_rdd)
some_df.printSchema()
some_df.show()

root
-- age: long (nullable = true)
-- name: string (nullable = true)

+---+-----+
age| name|
+---+-----+
 19| John|
 23|Smith|
 18|Sarah|
+---+-----+

In [0]:
# A dataframe is an RDD of rows plus information on the schema.
# performing **collect()* on either the RDD or the DataFrame gives the same result.
print(type(some_rdd),type(some_df))
some_df.show()

<class 'pyspark.rdd.RDD'> <class 'pyspark.sql.dataframe.DataFrame'>
+---+-----+
age| name|
+---+-----+
 19| John|
 23|Smith|
 18|Sarah|
+---+-----+

In [0]:
data = [("Java", 20000), ("Python", 100000), ("Scala", 3000)]
rdd = spark.sparkContext.parallelize(data)
dfFromRDD1 = spark.createDataFrame(rdd)
dfFromRDD1.printSchema()
dfFromRDD1.show()

dfFromRDD2 = rdd.toDF(["language","users_count"])
#dfFromRDD2.printSchema()
dfFromRDD2.show()

root
-- _1: string (nullable = true)
-- _2: long (nullable = true)

+------+------+
 _1| _2|
+------+------+
 Java| 20000|
Python|100000|
 Scala| 3000|
+------+------+

+--------+-----------+
language|users_count|
+--------+-----------+
 Java| 20000|
 Python| 100000|
 Scala| 3000|
+--------+-----------+

In [0]:
##data11 = [("Java"),("Python"),("CPP")]
data11 = [("Java",), ("Python",), ("Scala", )]
rdd11 = spark.sparkContext.parallelize(data11)
dfFromRDD11 = spark.createDataFrame(rdd11)
dfFromRDD11.printSchema()
dfFromRDD11.show()


root
-- _1: string (nullable = true)

+------+
 _1|
+------+
 Java|
Python|
 Scala|
+------+

In [0]:
from pyspark.sql.types import LongType
# In this case we create the dataframe from an RDD of tuples (rather than Rows) and provide the schema explicitly
another_rdd = sc.parallelize([("John", 19), ("Smith", 23), ("Sarah", 18)])
# Schema with two fields - person_name and person_age
schema = StructType([StructField("person_name", StringType(), False),
                     StructField("person_age", IntegerType(), False)])

# Create a DataFrame by applying the schema to the RDD and print the schema
another_df = sqlContext.createDataFrame(another_rdd, schema)
another_df.printSchema()
# root
#  |-- age: binteger (nullable = true)
#  |-- name: string (nullable = true)

root
-- person_name: string (nullable = false)
-- person_age: integer (nullable = false)

In [0]:
# when loading json files you can specify either a single file or a directory containing many json files.
path = "/FileStore/tables/people-1.json"

# Create a DataFrame from the file(s) pointed to by path
people_df = spark.read.json(path)
print('people is a',type(people_df))
# The inferred schema can be visualized using the printSchema() method.
people_df.show()

people_df.printSchema()

people is a <class 'pyspark.sql.dataframe.DataFrame'>
+----+-------+
 age| name|
+----+-------+
null|Michael|
 30| Andy|
 19| Justin|
+----+-------+

root
-- age: long (nullable = true)
-- name: string (nullable = true)

In [0]:
df2=people_df.select("name").where(people_df['name']=='Andy')
df2.show()

+----+
name|
+----+
Andy|
+----+

In [0]:
df2.write.parquet("/FileStore/tables/output5th")

In [0]:
%sh

ls -ltr /dbfs/FileStore/tables/output5th

total 0
-rw-r--r-- 1 root root 427 Sep 5 05:34 part-00000-tid-7458975689329521439-93777567-8571-4a5a-b669-3768195521eb-788-1-c000.snappy.parquet
-rw-r--r-- 1 root root 0 Sep 5 05:34 _started_7458975689329521439
-rw-r--r-- 1 root root 124 Sep 5 05:34 _committed_7458975689329521439
-rw-r--r-- 1 root root 0 Sep 5 05:34 _SUCCESS

In [0]:
# import pyspark class Row from module sql
from pyspark.sql import *

# Create Example Data - Departments and Employees

# Create the Departments
department1 = Row(id='123456', name='Computer Science')
department2 = Row(id='789012', name='Mechanical Engineering')
department3 = Row(id='345678', name='Theater and Drama')
department4 = Row(id='901234', name='Indoor Recreation')

# Create the Employees
Employee = Row("firstName", "lastName", "email", "salary")
employee1 = Employee('michael', 'armbrust', 'no-reply@berkeley.edu', 100000)
employee2 = Employee('xiangrui', 'meng', 'no-reply@stanford.edu', 120000)
employee3 = Employee('matei', None, 'no-reply@waterloo.edu', 140000)
employee4 = Employee(None, 'wendell', 'no-reply@berkeley.edu', 160000)
employee5 = Employee('michael', 'jackson', 'no-reply@neverla.nd', 80000)

# Create the DepartmentWithEmployees instances from Departments and Employees
departmentWithEmployees1 = Row(department=department1, employees=[employee1, employee2])
departmentWithEmployees2 = Row(department=department2, employees=[employee3, employee4])
departmentWithEmployees3 = Row(department=department3, employees=[employee5, employee4])
departmentWithEmployees4 = Row(department=department4, employees=[employee2, employee3])


In [0]:
departmentsWithEmployeesSeq1 = [departmentWithEmployees1, departmentWithEmployees2]
df1 = spark.createDataFrame(departmentsWithEmployeesSeq1)

display(df1)

departmentsWithEmployeesSeq2 = [departmentWithEmployees3, departmentWithEmployees4]
df2 = spark.createDataFrame(departmentsWithEmployeesSeq2)

display(df2)

department,employees
"List(345678, Theater and Drama)","List(List(michael, jackson, no-reply@neverla.nd, 80000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))"


In [0]:
df2.select("department.name").show()

+-----------------+
 name|
+-----------------+
Theater and Drama|
Indoor Recreation|
+-----------------+

In [0]:
unionDF = df1.union(df2)
display(unionDF)

department,employees
"List(123456, Computer Science)","List(List(michael, armbrust, no-reply@berkeley.edu, 100000), List(xiangrui, meng, no-reply@stanford.edu, 120000))"
"List(789012, Mechanical Engineering)","List(List(matei, null, no-reply@waterloo.edu, 140000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(345678, Theater and Drama)","List(List(michael, jackson, no-reply@neverla.nd, 80000), List(null, wendell, no-reply@berkeley.edu, 160000))"
"List(901234, Indoor Recreation)","List(List(xiangrui, meng, no-reply@stanford.edu, 120000), List(matei, null, no-reply@waterloo.edu, 140000))"


In [0]:
from pyspark.sql.functions import explode

explodeDF = unionDF.select(explode("employees.firstName"))
explodeDF.show(2,truncate= False)
unexplodeDF = unionDF.select("employees.firstName")
unexplodeDF.show(2,truncate= False)
#unionDF.printSchema()
#explodeDF.printSchema()
#unexplodeDF.printSchema()

+--------+
col |
+--------+
michael |
xiangrui|
+--------+
only showing top 2 rows

+-------------------+
firstName |
+-------------------+
[michael, xiangrui]|
[matei,] |
+-------------------+
only showing top 2 rows

In [0]:
filterDF = flattenDF.select("email").filter(flattenDF.firstName == "xiangrui").sort(flattenDF.lastName)
display(filterDF)

email
no-reply@stanford.edu
no-reply@stanford.edu


In [0]:
from pyspark.sql.functions import col, asc
whereDF = flattenDF.select("email").where((col("firstName") == "xiangrui") | (col("firstName") == "michael")).sort(asc("lastName"))
display(whereDF)

email
no-reply@berkeley.edu
no-reply@neverla.nd
no-reply@stanford.edu
no-reply@stanford.edu


In [0]:
from pyspark.sql.functions import countDistinct

countDistinctDF = whereDF.select("firstName", "lastName")\
  .groupBy("firstName")\
  .agg(countDistinct("lastName").alias("distinct_last_names"))

display(countDistinctDF)

firstName,distinct_last_names
xiangrui,1
michael,2
